In [1]:
# -*- coding: utf-8 -*-
# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.14.0
#   kernelspec:
#     display_name: Python 3 (ipykernel)
#     language: python
#     name: python3
# ---

# # 단계별 백테스트 디버깅 노트 (Vectorbt 기반)

# ## 1. 라이브러리 및 함수 임포트

import pandas as pd
import numpy as np
import yfinance as yf
import vectorbt as vbt
import json
import os
import logging
from datetime import datetime
import sys

# --- ensure_dir 함수 직접 정의 ---
def ensure_dir(directory_path: str):
    """주어진 경로의 디렉토리가 없으면 생성합니다."""
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        print(f"Created directory: {directory_path}")
# ---------------------------------

# 프로젝트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '.')) # 현재 작업 디렉토리를 루트로 가정, 필요시 수정
if project_root not in sys.path:
    sys.path.insert(0, project_root) # sys.path의 맨 앞에 추가
    print(f"Added project root to sys.path: {project_root}")
else:
    print(f"Project root already in sys.path: {project_root}")

# 필요한 함수 임포트
# utils.common 에 있는 함수들 (필요시 사용)
try:
    from src.utils.common import calculate_rsi as calculate_rsi_common, calculate_sma as calculate_sma_common
    print("Successfully imported common functions from src.utils.common")
except ImportError as e:
    print(f"Note: Could not import from src.utils.common: {e}")

# utils.strategy_utils 에 있는 함수들
try:
    from src.utils.strategy_utils import find_volume_breakout_pullback_nb
    from src.utils.strategy_utils import calculate_rsi, calculate_sma # strategy_utils의 함수 우선 사용
    print("Successfully imported functions from src.utils.strategy_utils")
except ImportError as e:
    print(f"CRITICAL ERROR: Could not import from src.utils.strategy_utils: {e}")
    # 필수 함수 없으면 진행 불가
    raise SystemExit("Missing critical functions from strategy_utils.py")

# backtesting 관련 함수 임포트
try:
    from src.backtesting.backtest_runner_vbt import run_vectorbt_backtest
    print("Successfully imported backtesting functions from src.backtesting.backtest_runner_vbt")
except ImportError as e:
    print(f"Error importing from src.backtesting.backtest_runner_vbt: {e}")
    # run_vectorbt_backtest 없으면 디버깅 대상 함수가 없는 것
    raise SystemExit("Missing run_vectorbt_backtest function.")


# 결과 저장 함수 임포트 (없으므로 None 할당)
save_results = None
print("save_results function will use direct JSON saving logic.")

# 로깅 설정
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', handlers=[logging.StreamHandler()])
logger = logging.getLogger(__name__) # 현재 모듈(노트북) 로거

print("\n라이브러리 및 함수 임포트 시도 완료")
print(f"Vectorbt version: {vbt.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

# 임포트된 함수 확인 (디버깅 목적)
print("\nChecking imported functions:")
print(f"ensure_dir defined: {callable(ensure_dir)}")
try:
    print(f"calculate_rsi (strategy_utils) defined: {callable(calculate_rsi)}")
    print(f"calculate_sma (strategy_utils) defined: {callable(calculate_sma)}")
    print(f"find_volume_breakout_pullback_nb defined: {callable(find_volume_breakout_pullback_nb)}")
    print(f"run_vectorbt_backtest defined: {callable(run_vectorbt_backtest)}")
except NameError as ne:
    print(f"Function check failed: {ne}")
print(f"save_results defined: {callable(save_results)}") # save_results 는 None




Project root already in sys.path: /Users/hjkim/Dev/Hjkim/Trading
Successfully imported common functions from src.utils.common
Successfully imported functions from src.utils.strategy_utils


2025-04-17 14:23:09,818 - INFO - [src.config.settings] load_dotenv() executed. Found .env file: True
2025-04-17 14:23:09,822 - INFO - [src.config.settings] Settings instance created successfully. Type: <class 'src.config.settings.Settings'>, ID: 13931717536
2025-04-17 14:23:09,823 - INFO - [src.config.settings] settings.CHALLENGE_SMA_PERIOD = 7


Successfully imported backtesting functions from src.backtesting.backtest_runner_vbt
save_results function will use direct JSON saving logic.

라이브러리 및 함수 임포트 시도 완료
Vectorbt version: 0.27.2
Pandas version: 1.5.3
Numpy version: 1.26.4

Checking imported functions:
ensure_dir defined: True
calculate_rsi (strategy_utils) defined: True
calculate_sma (strategy_utils) defined: True
find_volume_breakout_pullback_nb defined: True
run_vectorbt_backtest defined: True
save_results defined: False


In [2]:
# ## 2. 백테스트 파라미터 정의

# 실패했던 특정 조합의 파라미터 설정 (run_robustness_test.py 참고)
symbol = "BTC-USD"
start_date = "2023-07-01"
end_date = "2023-12-31"
timeframe = "1h" # yfinance에서 지원하는 형식

# 가변 파라미터 (실패했던 특정 값으로 설정)
params = {
    'pullback_threshold': 0.05,
    'rsi_period': 10,
    'atr_sl_multiplier': 1.5, # SL/TP 계산 시 사용
    'atr_tp_multiplier': 2.0, # SL/TP 계산 시 사용
    # strategy_utils.calculate_sma 를 위해 추가 (없으면 기본값 사용)
    'sma_short_period': 7,
    'sma_long_period': 20,
}

# 고정 파라미터 (run_robustness_test.py 또는 run_mcp_experiment.py 참고)
fixed_params = {
    'breakout_lookback': 20,
    'pullback_lookback': 5,
    'volume_lookback': 10, # Volume Breakout 함수 및 Volume SMA 계산 시 사용
    'volume_multiplier': 2.0, # Volume Breakout 함수 시 사용
    'atr_window': 10, # ATR 계산 시 사용
}

# 모든 파라미터 결합
# run_vectorbt_backtest 함수 시그니처에 맞춰 필요한 파라미터만 포함하는 것이 좋음
# 여기서는 디버깅 편의상 모두 합침
all_params = {**params, **fixed_params}

# 결과 저장 경로 설정
results_dir = os.path.join(project_root, 'mcp', 'results_debug') # 디버깅용 결과 폴더
ensure_dir(results_dir)
# 파일명 생성 (간단하게)
param_str = "_".join([f"{k}{v}" for k, v in sorted(params.items())]) # 가변 파라미터 기준
result_filename_base = f"debug_{symbol}_{start_date}_{end_date}_{timeframe}_{param_str}"
result_filepath = os.path.join(results_dir, f"{result_filename_base}.json")

print("파라미터 정의 완료:")
print(f"Symbol: {symbol}")
print(f"Period: {start_date} to {end_date} ({timeframe})")
print("All Parameters (for debugging):")
for key, value in all_params.items():
    print(f"  {key}: {value}")
print(f"Result File Path: {result_filepath}")




파라미터 정의 완료:
Symbol: BTC-USD
Period: 2023-07-01 to 2023-12-31 (1h)
All Parameters (for debugging):
  pullback_threshold: 0.05
  rsi_period: 10
  atr_sl_multiplier: 1.5
  atr_tp_multiplier: 2.0
  sma_short_period: 7
  sma_long_period: 20
  breakout_lookback: 20
  pullback_lookback: 5
  volume_lookback: 10
  volume_multiplier: 2.0
  atr_window: 10
Result File Path: /Users/hjkim/Dev/Hjkim/Trading/mcp/results_debug/debug_BTC-USD_2023-07-01_2023-12-31_1h_atr_sl_multiplier1.5_atr_tp_multiplier2.0_pullback_threshold0.05_rsi_period10_sma_long_period20_sma_short_period7.json


In [3]:
# ## 3. 데이터 로딩 (yfinance)

logger.info(f"데이터 로딩 시작: {symbol}, {start_date} to {end_date}, interval={timeframe}")
price_data = None # 초기화
try:
    price_data = yf.download(
        symbol,
        start=start_date,
        end=end_date,
        interval=timeframe,
        progress=True
    )
    if price_data.empty:
        logger.error("yf.download() returned an empty DataFrame.")
        raise ValueError("데이터 로딩 실패: 빈 데이터프레임 반환")
    else:
        logger.debug(f"yf.download raw data shape: {price_data.shape}")
        logger.debug(f"yf.download raw columns type: {type(price_data.columns)}") # 컬럼 타입 확인
        logger.debug(f"yf.download raw columns: {price_data.columns}") # 원본 컬럼 확인

        # --- MultiIndex 처리 추가 ---
        if isinstance(price_data.columns, pd.MultiIndex):
            logger.warning("DataFrame columns are MultiIndex. Attempting to flatten...")
            # 가정: 두 번째 레벨이 티커명 등 불필요한 레벨이라고 가정하고 제거 (일반적 케이스)
            try:
                price_data.columns = price_data.columns.droplevel(1) # level 1 (두 번째 레벨) 제거
                logger.info(f"Successfully flattened MultiIndex. New columns: {price_data.columns.tolist()}")
            except Exception as multi_idx_e:
                logger.error(f"Failed to automatically flatten MultiIndex: {multi_idx_e}")
                logger.error(f"MultiIndex structure: {price_data.columns}")
                raise ValueError("Could not handle MultiIndex columns structure.")
        # --------------------------

        # yfinance가 timezone을 붙여 반환하는 경우가 있으므로 제거
        if price_data.index.tz is not None:
            logger.debug("Removing timezone information from index.")
            price_data.index = price_data.index.tz_localize(None)

        # 컬럼명 표준화 (소문자) - MultiIndex 처리 후에 실행
        price_data.columns = price_data.columns.str.lower()
        logger.info(f"Columns after converting to lowercase: {price_data.columns.tolist()}")

        required_cols = ['open', 'high', 'low', 'close', 'volume']
        missing_cols = [col for col in required_cols if col not in price_data.columns]
        if missing_cols:
             logger.error(f"Missing required columns AFTER lowercase conversion: {missing_cols}. Available columns: {price_data.columns.tolist()}")
             raise ValueError(f"Missing required columns after lowercase conversion: {missing_cols}")

        logger.info(f"데이터 로딩 및 컬럼 확인 성공. Shape: {price_data.shape}")
        print("\n--- 데이터 정보 (컬럼 변환 후) ---")
        price_data.info()
        print("\n--- 결측치 확인 (컬럼 변환 후) ---")
        print(price_data.isnull().sum())

        # 결측치 처리 (ffill 후 bfill)
        initial_nan_count = price_data.isnull().sum().sum()
        if initial_nan_count > 0:
            logger.warning(f"원본 데이터에 {initial_nan_count}개의 결측치 발견. ffill()/bfill() 적용.")
            price_data = price_data.ffill().bfill()
            remaining_nan_count = price_data.isnull().sum().sum()
            if remaining_nan_count > 0:
                 raise ValueError(f"ffill/bfill 후에도 {remaining_nan_count}개의 결측치 남음.")
            else:
                 logger.info("결측치 처리 완료.")
            print("\n--- 결측치 처리 후 정보 ---")
            price_data.info()
            print(price_data.isnull().sum())
        else:
            logger.info("원본 데이터에 결측치 없음.")

except Exception as e:
    logger.exception(f"데이터 로딩 또는 전처리 중 오류 발생: {e}")


# 데이터 로딩 실패 시 중단
if price_data is None or price_data.empty:
     logger.critical("데이터 로딩 실패로 디버깅 중단")
     raise SystemExit("데이터 로딩 실패")

2025-04-17 14:23:09,837 - INFO - 데이터 로딩 시작: BTC-USD, 2023-07-01 to 2023-12-31, interval=1h


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
2025-04-17 14:23:10,685 - WARNING - DataFrame columns are MultiIndex. Attempting to flatten...
2025-04-17 14:23:10,686 - INFO - Successfully flattened MultiIndex. New columns: ['Close', 'High', 'Low', 'Open', 'Volume']
2025-04-17 14:23:10,686 - INFO - Columns after converting to lowercase: ['close', 'high', 'low', 'open', 'volume']
2025-04-17 14:23:10,686 - INFO - 데이터 로딩 및 컬럼 확인 성공. Shape: (4391, 5)
2025-04-17 14:23:10,690 - INFO - 원본 데이터에 결측치 없음.



--- 데이터 정보 (컬럼 변환 후) ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4391 entries, 2023-07-01 00:00:00 to 2023-12-30 23:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   close   4391 non-null   float64
 1   high    4391 non-null   float64
 2   low     4391 non-null   float64
 3   open    4391 non-null   float64
 4   volume  4391 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 205.8 KB

--- 결측치 확인 (컬럼 변환 후) ---
Price
close     0
high      0
low       0
open      0
volume    0
dtype: int64


In [4]:
# ## 4. 지표 계산 및 확인

logger.info("지표 계산 시작")
indicators_df = None # 초기화
try:
    # price_data는 이미 검증 및 전처리 되었다고 가정

    # --- strategy_utils 함수 사용 ---
    # SMA 계산
    sma_short_period = all_params.get('sma_short_period', 7)
    sma_long_period = all_params.get('sma_long_period', 20)
    # calculate_sma는 원본을 수정하므로 copy() 사용
    price_data_copy = price_data.copy()
    price_data_copy = calculate_sma(price_data_copy, window=sma_short_period, feature_col='close')
    price_data_copy = calculate_sma(price_data_copy, window=sma_long_period, feature_col='close')
    logger.debug(f"SMA 계산 완료 (Short: {sma_short_period}, Long: {sma_long_period}) using strategy_utils.")

    # RSI 계산
    rsi_period = all_params['rsi_period']
    price_data_copy = calculate_rsi(price_data_copy, window=rsi_period, feature_col='close')
    logger.debug(f"RSI 계산 완료 (Period: {rsi_period}) using strategy_utils.")

    # --- ATR 계산 (vectorbt 내장 사용) ---
    atr_window = all_params['atr_window']
    atr_indicator = vbt.ATR.run(price_data_copy['high'], price_data_copy['low'], price_data_copy['close'], window=atr_window)
    atr = atr_indicator.atr # ATR 값 Series 추출
    logger.debug(f"ATR 계산 완료 (Window: {atr_window}) using vectorbt.")
    price_data_copy['ATR'] = atr # 데이터프레임에 추가

    # --- 거래량 이동평균 계산 (strategy_utils 사용) ---
    volume_lookback = all_params['volume_lookback']
    price_data_copy = calculate_sma(price_data_copy, window=volume_lookback, feature_col='volume')
    volume_sma_col_name = f'SMA_{volume_lookback}' # strategy_utils 에서 생성된 컬럼명
    logger.debug(f"Volume SMA 계산 완료 (Window: {volume_lookback}).")
    # 컬럼명을 일관성 있게 변경 (옵션)
    if volume_sma_col_name in price_data_copy.columns:
        price_data_copy.rename(columns={volume_sma_col_name: 'Volume_SMA'}, inplace=True)


    # 최종 indicators_df 로 할당
    indicators_df = price_data_copy

    print("\n--- 통합 지표 데이터프레임 (Head) ---")
    # lookback 기간 중 가장 큰 값 + 5
    print(indicators_df.head(max(rsi_period, atr_window, volume_lookback, sma_long_period) + 5))
    print("\n--- 통합 지표 데이터프레임 (Tail) ---")
    print(indicators_df.tail())
    print("\n--- 통합 지표 결측치 ---")
    # 지표 계산 초반 NaN은 정상
    print(indicators_df.isnull().sum())
    print("\n--- 지표 통계 (NaN 제외) ---")
    print(indicators_df.describe())

    # 지표 계산 후에도 NaN 이 많이 남아있는지 확인 (특히 후반부)
    if indicators_df.tail().isnull().sum().sum() > 0:
        logger.warning("지표 계산 후 데이터 후반부에 NaN 값이 존재합니다.")
        print(indicators_df.tail().isnull().sum())


except Exception as e:
    logger.exception(f"지표 계산 중 오류 발생: {e}")


if indicators_df is None or indicators_df.empty:
    logger.critical("지표 계산 실패 또는 결과 없음으로 디버깅 중단")
    raise SystemExit("지표 계산 실패")







2025-04-17 14:23:10,696 - INFO - 지표 계산 시작



--- 통합 지표 데이터프레임 (Head) ---
Price                       close          high           low          open  \
Datetime                                                                      
2023-07-01 00:00:00  30477.494141  30524.464844  30432.773438  30471.847656   
2023-07-01 01:00:00  30438.726562  30478.087891  30410.515625  30466.806641   
2023-07-01 02:00:00  30419.037109  30514.943359  30399.609375  30440.601562   
2023-07-01 03:00:00  30386.820312  30435.925781  30380.173828  30430.699219   
2023-07-01 04:00:00  30380.412109  30413.599609  30328.865234  30388.476562   
2023-07-01 05:00:00  30425.517578  30425.517578  30382.873047  30384.189453   
2023-07-01 06:00:00  30444.800781  30456.429688  30402.423828  30423.689453   
2023-07-01 07:00:00  30457.009766  30457.009766  30407.041016  30444.335938   
2023-07-01 08:00:00  30441.486328  30469.488281  30426.953125  30458.746094   
2023-07-01 09:00:00  30437.025391  30461.824219  30431.921875  30437.552734   
2023-07-01 10:00:00  30

In [5]:
# ## 5. 진입/청산 시그널 생성 확인

logger.info("시그널 생성 시작")
entries = None
exits = None
try:
    # --- Numba 함수 (find_volume_breakout_pullback_nb) 호출 ---
    logger.debug("Numba 시그널 함수 호출 준비 (find_volume_breakout_pullback_nb)...")

    # 필요한 입력 배열 준비 (NaN 처리 중요)
    # indicators_df 에는 이전 단계에서 ffill/bfill 처리된 데이터가 있어야 함
    # Numba 함수는 NaN 처리 못하므로, 여기서 다시 확인/처리
    required_cols_for_signal = ['high', 'low', 'close', 'volume'] # + RSI?
    if indicators_df[required_cols_for_signal].isnull().any().any():
        logger.warning("시그널 생성 전 입력 데이터에 NaN 발견. ffill/bfill 재적용.")
        indicators_df[required_cols_for_signal] = indicators_df[required_cols_for_signal].ffill().bfill()
        if indicators_df[required_cols_for_signal].isnull().any().any():
             raise ValueError("NaN 처리 후에도 시그널 입력 데이터에 NaN 존재.")

    high_prices = indicators_df['high'].values
    low_prices = indicators_df['low'].values
    close_prices = indicators_df['close'].values
    volume = indicators_df['volume'].values
    # rsi_values = indicators_df['RSI'].fillna(50).values # 함수 시그니처에 RSI 필요시

    # 파라미터 값 추출
    breakout_lookback = all_params['breakout_lookback']
    pullback_lookback = all_params['pullback_lookback']
    pullback_threshold = all_params['pullback_threshold']
    volume_lookback_sig = all_params['volume_lookback']
    volume_multiplier = all_params['volume_multiplier']
    # rsi_period_sig = all_params['rsi_period']

    # Numba 시그널 함수 호출 (strategy_utils 버전)
    entry_signal_values = find_volume_breakout_pullback_nb(
        high_prices.astype(np.float64), # Numba는 타입 명시 선호
        low_prices.astype(np.float64),
        close_prices.astype(np.float64),
        volume.astype(np.float64),
        breakout_lookback,
        pullback_lookback,
        pullback_threshold,
        volume_lookback_sig,
        volume_multiplier
        # rsi_values=rsi_values, # 함수 시그니처에 맞게 전달
        # rsi_period=rsi_period_sig
    )
    logger.info(f"Numba 시그널 함수 호출 완료. 반환 타입: {type(entry_signal_values)}, Shape: {getattr(entry_signal_values, 'shape', 'N/A')}")

    # 시그널 결과 확인 (배열 반환 가정)
    if isinstance(entry_signal_values, np.ndarray):
        # Numba 함수 반환값이 boolean 인지 int 인지 확인 필요
        # vectorbt는 boolean 타입 선호
        entries = pd.Series(entry_signal_values.astype(bool), index=indicators_df.index, name='Entries')
        exits = pd.Series(False, index=indicators_df.index, name='Exits') # 청산 신호 별도 없음
        print(f"총 진입 시그널 개수: {entries.sum()}")
    else:
         raise TypeError(f"예상치 못한 시그널 함수 반환 타입: {type(entry_signal_values)}")

    # 생성된 시그널 확인
    if entries is not None and entries.sum() > 0:
        print("진입 시그널 발생 인덱스 (최대 10개):")
        print(entries[entries == True].index[:10])
        first_entry_idx = entries[entries == True].index[0]
        print(f"\n첫 진입 시점 ({first_entry_idx})의 데이터 및 지표:")
        # 주요 지표만 출력
        print(indicators_df.loc[first_entry_idx, ['open', 'high', 'low', 'close', 'volume', 'RSI', 'ATR', 'Volume_SMA']])
    else:
        print("생성된 진입 시그널이 없습니다.")
        logger.warning("진입 시그널이 생성되지 않았습니다. 파라미터, 데이터, Numba 함수 로직 확인 필요.")

except Exception as e:
    logger.exception(f"시그널 생성 중 오류 발생: {e}")


# 진입 시그널이 없는 경우에도 다음 단계 진행 (결과 저장 시 'No entries' 기록)
if entries is None:
     logger.error("시그널 생성 중 오류 발생하여 entries가 None입니다.")
     # raise SystemExit("시그널 생성 실패") # 진행 중단 옵션


2025-04-17 14:23:10,961 - INFO - 시그널 생성 시작
2025-04-17 14:23:10,964 - INFO - Numba 시그널 함수 호출 완료. 반환 타입: <class 'numpy.ndarray'>, Shape: (4391,)


총 진입 시그널 개수: 717
진입 시그널 발생 인덱스 (최대 10개):
DatetimeIndex(['2023-07-02 23:00:00', '2023-07-03 00:00:00',
               '2023-07-03 01:00:00', '2023-07-03 02:00:00',
               '2023-07-03 03:00:00', '2023-07-03 16:00:00',
               '2023-07-03 17:00:00', '2023-07-03 18:00:00',
               '2023-07-03 19:00:00', '2023-07-03 20:00:00'],
              dtype='datetime64[ns]', name='Datetime', freq=None)

첫 진입 시점 (2023-07-02 23:00:00)의 데이터 및 지표:
Price
open          3.065471e+04
high          3.065471e+04
low           3.056308e+04
close         3.061972e+04
volume        2.562365e+08
RSI           5.692486e+01
ATR           1.144722e+02
Volume_SMA    2.770734e+08
Name: 2023-07-02 23:00:00, dtype: float64


In [6]:
# ## 6. Stop Loss / Take Profit 레벨 계산 확인

logger.info("SL/TP 레벨 계산 시작")
sl_stop = None
tp_stop = None
try:
    # run_vectorbt_backtest 에서 use_atr_exit = True 일 경우 ATR 기반 SL/TP 사용 가정
    # 여기서는 항상 ATR 기반으로 계산하여 디버깅
    atr_sl_multiplier = all_params['atr_sl_multiplier']
    atr_tp_multiplier = all_params['atr_tp_multiplier']

    # ATR 값 확인 (indicators_df['ATR'])
    if 'ATR' not in indicators_df.columns or indicators_df['ATR'].isnull().all():
        raise ValueError("ATR 값이 없거나 모두 NaN입니다. SL/TP 계산 불가.")

    # ATR 기반 절대값 변동폭 Series 계산
    sl_stop_series = indicators_df['ATR'] * atr_sl_multiplier
    tp_stop_series = indicators_df['ATR'] * atr_tp_multiplier

    # NaN 처리 (ATR 계산 초반 NaN)
    sl_stop_series = sl_stop_series.ffill().bfill()
    tp_stop_series = tp_stop_series.ffill().bfill()

    # vectorbt Portfolio.from_signals 에 전달할 Series
    sl_stop = sl_stop_series
    tp_stop = tp_stop_series

    # 결과 확인
    sl_tp_df = pd.DataFrame({
        'ATR': indicators_df['ATR'],
        'SL_Stop_Value': sl_stop,
        'TP_Stop_Value': tp_stop
    })
    print("\n--- SL/TP 계산 결과 (Tail) ---")
    print(sl_tp_df.tail())
    print("\n--- SL/TP 계산 결과 결측치 ---")
    print(sl_tp_df.isnull().sum()) # 0 이어야 함
    # SL/TP 값이 합리적인 범위 내에 있는지 확인 (0 이하 값 등)
    if (sl_stop <= 0).any() or (tp_stop <= 0).any():
        logger.warning(f"계산된 SL 또는 TP 값 중 0 이하인 경우가 {(sl_stop <= 0).sum() + (tp_stop <= 0).sum()}건 있습니다.")
        print("0 이하 SL/TP 값 샘플:")
        print(sl_tp_df[(sl_tp_df['SL_Stop_Value'] <= 0) | (sl_tp_df['TP_Stop_Value'] <= 0)].head())

except Exception as e:
    logger.exception(f"SL/TP 계산 중 오류 발생: {e}")


if sl_stop is None or tp_stop is None:
    logger.critical("SL/TP 계산 실패로 디버깅 중단")
    raise SystemExit("SL/TP 계산 실패")




2025-04-17 14:23:11,002 - INFO - SL/TP 레벨 계산 시작



--- SL/TP 계산 결과 (Tail) ---
                            ATR  SL_Stop_Value  TP_Stop_Value
Datetime                                                     
2023-12-30 19:00:00  176.579315     264.868972     353.158630
2023-12-30 20:00:00  173.962621     260.943932     347.925242
2023-12-30 21:00:00  156.659048     234.988572     313.318096
2023-12-30 22:00:00  158.699733     238.049599     317.399465
2023-12-30 23:00:00  157.487991     236.231987     314.975983

--- SL/TP 계산 결과 결측치 ---
ATR              9
SL_Stop_Value    0
TP_Stop_Value    0
dtype: int64


In [7]:
# ## 6.1 SL/TP 및 ATR 값 상세 분석 (추가)

if 'sl_tp_df' in locals() and sl_tp_df is not None:
    logger.info("ATR 및 SL/TP 값 상세 분석 시작...")

    print("\n--- ATR 및 SL/TP 값 통계 ---")
    # 주요 통계량 출력 (0 값이나 극단적인 값 확인)
    print(sl_tp_df.describe())

    # ATR 값이 0 이거나 음수인 경우가 있는지 확인
    atr_non_positive_count = (indicators_df['ATR'] <= 0).sum()
    if atr_non_positive_count > 0:
        logger.warning(f"ATR 값 중 0 이하인 경우가 {atr_non_positive_count}건 있습니다.")
        print("\n--- 0 이하 ATR 값 샘플 ---")
        print(indicators_df[indicators_df['ATR'] <= 0].head())

    # SL/TP 값이 0 이하인 경우 재확인 (이전 셀에서도 확인했지만 상세 로깅 추가)
    sl_tp_non_positive_count = ((sl_stop <= 0) | (tp_stop <= 0)).sum()
    if sl_tp_non_positive_count > 0:
         logger.warning(f"계산된 SL 또는 TP 값 중 0 이하인 경우가 {sl_tp_non_positive_count}건 있습니다.")
         # 이전에 이미 출력했으므로 여기서는 로그만 남김

    # --- 시각화: 가격 대비 ATR 비율 확인 ---
    # 가격 대비 ATR 비율이 너무 작으면 SL/TP가 거의 발동하지 않을 수 있음
    try:
        logger.info("가격 대비 ATR 비율 시각화 시도...")
        relative_atr = (indicators_df['ATR'] / indicators_df['close']) * 100 # ATR을 종가의 %로 표시
        relative_atr.name = 'Relative ATR (%)'

        # Plotly를 사용하여 시각화 (별도 플롯)
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots

        fig_atr = make_subplots(rows=2, cols=1, shared_xaxes=True,
                                subplot_titles=('Close Price', 'Relative ATR (%)'),
                                row_heights=[0.7, 0.3])

        fig_atr.add_trace(go.Scatter(x=indicators_df.index, y=indicators_df['close'], name='Close'), row=1, col=1)
        fig_atr.add_trace(go.Scatter(x=relative_atr.index, y=relative_atr, name='Relative ATR (%)'), row=2, col=1)

        fig_atr.update_layout(title='Price vs Relative ATR', height=600)
        fig_atr.show()
        logger.info("가격 대비 ATR 비율 플롯 생성 완료.")

        # 상대 ATR 값 통계 확인
        print("\n--- 가격 대비 상대 ATR (%) 통계 ---")
        print(relative_atr.describe())

    except Exception as e:
        logger.exception(f"ATR 시각화 중 오류 발생: {e}")

else:
    logger.warning("SL/TP 데이터프레임(sl_tp_df)이 없어 상세 분석을 건너<0xEB><0x9B><0x84>니다.")


2025-04-17 14:23:12,396 - INFO - ATR 및 SL/TP 값 상세 분석 시작...
2025-04-17 14:23:12,402 - INFO - 가격 대비 ATR 비율 시각화 시도...



--- ATR 및 SL/TP 값 통계 ---
               ATR  SL_Stop_Value  TP_Stop_Value
count  4382.000000    4391.000000    4391.000000
mean    140.965292     211.193688     281.591584
std      90.999542     136.474707     181.966277
min      13.656838      20.485258      27.313677
25%      74.784055     111.825543     149.100724
50%     118.794293     177.619238     236.825651
75%     183.411798     275.098836     366.798448
max     796.717760    1195.076639    1593.435519


2025-04-17 14:23:12,519 - INFO - 가격 대비 ATR 비율 플롯 생성 완료.



--- 가격 대비 상대 ATR (%) 통계 ---
count    4382.000000
mean        0.425059
std         0.241224
min         0.046474
25%         0.256149
50%         0.384466
75%         0.541262
max         2.309178
Name: Relative ATR (%), dtype: float64


In [8]:
# ## 7. Vectorbt 포트폴리오 실행 및 결과 확인 (ATR 비율 SL/TP 테스트)

logger.info("Vectorbt 포트폴리오 시뮬레이션 시작 (ATR 비율 SL/TP 테스트)")
portfolio = None
stats_dict = None
result_data = None # 최종 결과 저장용

# 진입 시그널과 SL/TP Series (절대값)가 준비되었다고 가정
if 'entries' in locals() and entries is not None and entries.sum() > 0 and \
   'sl_stop' in locals() and sl_stop is not None and \
   'tp_stop' in locals() and tp_stop is not None:
    try:
        # 가격 데이터 준비
        close_prices_vbt = indicators_df['close']

        # --- SL/TP를 가격 대비 비율 Series로 변환 ---
        # sl_stop, tp_stop은 ## 6. 에서 계산된 절대 ATR 변동폭 Series
        # 0으로 나누는 것을 방지하기 위해 작은 값(epsilon) 추가 또는 0인 가격 처리 필요
        epsilon = 1e-9
        # .abs()를 추가하여 혹시 모를 음수 가격/ATR 처리 강화 (보통은 없어야 함)
        sl_stop_ratio_series = (sl_stop.abs() / (close_prices_vbt.abs() + epsilon)).fillna(0) # 비율 계산 후 NaN은 0으로 채움
        tp_stop_ratio_series = (tp_stop.abs() / (close_prices_vbt.abs() + epsilon)).fillna(0)
        logger.debug("Calculated SL/TP ratio series.")
        print("\n--- SL/TP Ratio Series (Tail) ---")
        # 비율이 너무 크거나 작지 않은지 확인 (예: 0 ~ 1 사이여야 함)
        print(pd.DataFrame({'SL_Ratio': sl_stop_ratio_series, 'TP_Ratio': tp_stop_ratio_series}).describe())
        print(pd.DataFrame({'SL_Ratio': sl_stop_ratio_series, 'TP_Ratio': tp_stop_ratio_series}).tail())
        # -------------------------------------------

        logger.debug(f"Calling vbt.Portfolio.from_signals with freq='{timeframe}' and ATR Ratio SL/TP...")
        init_cash = 10000.0
        commission = 0.001
        slippage = 0.001

        portfolio = vbt.Portfolio.from_signals(
            close=close_prices_vbt,
            entries=entries,
            exits=exits if exits is not None else pd.Series(False, index=entries.index),
            freq=timeframe,
            # --- 비율 Series 전달 ---
            sl_stop=sl_stop_ratio_series, # 가격 대비 비율 Series
            tp_stop=tp_stop_ratio_series, # 가격 대비 비율 Series
            # -----------------------
            init_cash=init_cash,
            fees=commission,
            slippage=slippage,
        )

        logger.info("포트폴리오 시뮬레이션 완료")
        print("\n--- 포트폴리오 요약 (ATR 비율 SL/TP) ---")
        try:
            print(f"Initial Cash: {portfolio.init_cash:.2f}")
            print(f"Final Value: {portfolio.final_value():.2f}")
            print(f"Total Return [%]: {portfolio.total_return() * 100:.2f}")
            print(f"Max Drawdown [%]: {portfolio.max_drawdown() * 100:.2f}")
            try:
                 print(f"Win Rate [%]: {portfolio.win_rate() * 100:.2f}")
            except AttributeError:
                 print("Win Rate [%]: N/A (AttributeError)")
            print(f"Sharpe Ratio: {portfolio.sharpe_ratio():.2f}")
            print(f"Total Trades: {portfolio.trades.count()}")
            print(f"Total Closed Trades: {portfolio.trades.closed.count()}") # 중요!
        except Exception as summary_e:
            logger.warning(f"포트폴리오 요약 정보 출력 중 오류: {summary_e}")


        # 거래 내역 확인
        try:
            trades = portfolio.trades.records_readable
            print(f"\n--- 총 거래 횟수: {len(trades)} ---")
            if not trades.empty:
                print("거래 내역 (최대 10개):")
                print(trades.head(10))
                closed_trades = trades[trades['Status'] == 'Closed']
                print(f"\n--- 종료된 거래 횟수 (from records): {len(closed_trades)} ---") # 중요!
                if not closed_trades.empty:
                    print("종료된 거래 샘플:")
                    print(closed_trades.head())
            else:
                print("실행된 거래가 없습니다.")
                logger.warning("포트폴리오 시뮬레이션 결과, 거래가 실행되지 않았습니다.")
        except Exception as trades_e:
            logger.warning(f"거래 내역 확인 중 오류: {trades_e}")


        # 통계 계산
        logger.debug("포트폴리오 통계 계산 시도...")
        if portfolio.trades.closed.count() > 0:
             stats_dict_raw = portfolio.stats()
             stats_dict = {}
             for k, v in stats_dict_raw.items():
                 # JSON 직렬화 처리
                 if isinstance(v, (np.generic, np.ndarray)):
                     try: stats_dict[k] = v.item() if v.ndim == 0 else v.tolist()
                     except ValueError: stats_dict[k] = v.tolist()
                 elif isinstance(v, (pd.Timestamp, pd.Timedelta)): stats_dict[k] = str(v)
                 elif pd.isna(v): stats_dict[k] = None
                 elif isinstance(v, (float, int, str, bool)) or v is None: stats_dict[k] = v
                 else:
                     logger.warning(f"통계 결과 직렬화 불가 타입: Key='{k}', Type='{type(v)}'. 문자열 변환.")
                     stats_dict[k] = str(v)

             logger.info("포트폴리오 통계 계산 및 기본 타입 변환 완료.")
             print("\n--- 포트폴리오 통계 (JSON 변환 가능 타입) ---")
             print(json.dumps(stats_dict, indent=2, default=str))

             result_data = {
                 'test_type': 'atr_ratio_sl_tp',
                 'symbol': symbol,
                 'start_date': start_date,
                 'end_date': end_date,
                 'timeframe': timeframe,
                 'parameters': all_params,
                 'metrics': stats_dict,
                 'run_timestamp': datetime.now().isoformat()
             }
        else:
             logger.warning("종료된 거래가 없어 상세 통계를 계산할 수 없습니다.")
             stats_dict = {'message': 'No closed trades with ATR ratio SL/TP'}
             result_data = {
                 'test_type': 'atr_ratio_sl_tp',
                 'symbol': symbol,
                 'start_date': start_date,
                 'end_date': end_date,
                 'timeframe': timeframe,
                 'parameters': all_params,
                 'metrics': {
                     'Start Value': portfolio.init_cash,
                     'End Value': portfolio.final_value(),
                     'Total Return [%]': portfolio.total_return() * 100,
                     'Total Trades': portfolio.trades.count(),
                     'Total Closed Trades': 0,
                     'message': 'No trades were closed with ATR ratio SL/TP.'
                     },
                 'run_timestamp': datetime.now().isoformat()
             }

    except Exception as e:
        logger.exception(f"Vectorbt 포트폴리오 실행 또는 통계 계산 중 오류 발생 (ATR 비율 SL/TP): {e}")
        result_data = {
             'test_type': 'atr_ratio_sl_tp',
             'symbol': symbol,
             'start_date': start_date,
             'end_date': end_date,
             'timeframe': timeframe,
             'parameters': all_params,
             'metrics': {'error': f'Portfolio execution/stats error (ATR ratio SL/TP): {str(e)}'},
             'run_timestamp': datetime.now().isoformat()
         }

# 시그널 없음 / 오류 처리
elif entries is not None and entries.sum() == 0:
    logger.error("진입 시그널이 없어 포트폴리오 시뮬레이션을 건너<0xEB><0x9B><0x84>니다.")
    result_data = {
         'test_type': 'atr_ratio_sl_tp',
         'symbol': symbol,
         'start_date': start_date,
         'end_date': end_date,
         'timeframe': timeframe,
         'parameters': all_params,
         'metrics': {'message': 'No entry signals generated'},
         'run_timestamp': datetime.now().isoformat()
     }
else:
     logger.error("시그널 생성 오류 또는 SL/TP 데이터 누락으로 포트폴리오 시뮬레이션을 건너<0xEB><0x9B><0x84>니다.")
     result_data = {
         'test_type': 'atr_ratio_sl_tp',
         'symbol': symbol,
         'start_date': start_date,
         'end_date': end_date,
         'timeframe': timeframe,
         'parameters': all_params,
         'metrics': {'error': 'Signal generation failed or SL/TP data missing'},
         'run_timestamp': datetime.now().isoformat()
     }

# 결과 확인
if result_data:
    print("\n--- 백테스트 결과 (ATR 비율 SL/TP) ---")
    print(json.dumps(result_data.get('metrics', {}), indent=2, default=str))
else:
    print("\n--- 백테스트 결과 없음 (ATR 비율 SL/TP) ---")

2025-04-17 14:24:56,984 - INFO - Vectorbt 포트폴리오 시뮬레이션 시작 (고정 SL/TP 테스트)
2025-04-17 14:24:58,424 - INFO - 포트폴리오 시뮬레이션 완료



--- 포트폴리오 요약 (고정 SL/TP) ---
Initial Cash: 10000.00
Final Value: 9775.15
Total Return [%]: -2.25
Max Drawdown [%]: -18.69
Win Rate [%]: N/A (AttributeError)
Sharpe Ratio: -0.04
Total Trades: 82
Total Closed Trades: 82

--- 총 거래 횟수: 82 ---
거래 내역 (최대 10개):
   Exit Trade Id  Column      Size     Entry Timestamp  Avg Entry Price  \
0              0       0  0.325935 2023-07-02 23:00:00     30650.338469   
1              1       0  0.326510 2023-07-03 20:00:00     31164.545521   
2              2       0  0.324492 2023-07-06 07:00:00     30840.917529   
3              3       0  0.325879 2023-07-06 14:00:00     30196.042830   
4              4       0  0.325331 2023-07-10 20:00:00     30827.992508   
5              5       0  0.323971 2023-07-10 22:00:00     30366.820859   
6              6       0  0.324308 2023-07-13 16:00:00     30861.086115   
7              7       0  0.326667 2023-07-13 20:00:00     31372.030143   
8              8       0  0.329570 2023-07-16 15:00:00     30420.25901

2025-04-17 14:24:59,278 - INFO - 포트폴리오 통계 계산 및 기본 타입 변환 완료.



--- 포트폴리오 통계 (JSON 변환 가능 타입) ---
{
  "Start": "2023-07-01 00:00:00",
  "End": "2023-12-30 23:00:00",
  "Period": "182 days 23:00:00",
  "Start Value": 10000.0,
  "End Value": 9775.15259991759,
  "Total Return [%]": -2.248474000824099,
  "Benchmark Return [%]": 38.47291389515939,
  "Max Gross Exposure [%]": 100.0,
  "Total Fees Paid": 1614.0472183553582,
  "Max Drawdown [%]": 18.694343829364115,
  "Max Drawdown Duration": "102 days 01:00:00",
  "Total Trades": 82,
  "Total Closed Trades": 82,
  "Total Open Trades": 0,
  "Open Trade PnL": 0.0,
  "Win Rate [%]": 41.46341463414634,
  "Best Trade [%]": 6.700435117553328,
  "Worst Trade [%]": -2.851561944727251,
  "Avg Winning Trade [%]": 2.3238282174492593,
  "Avg Losing Trade [%]": -1.656108871084026,
  "Avg Winning Trade Duration": "1 days 08:17:38.823529411",
  "Avg Losing Trade Duration": "1 days 03:01:15",
  "Profit Factor": 0.9713487650810645,
  "Expectancy": -2.7420414644196143,
  "Sharpe Ratio": -0.04102807864327327,
  "Calmar Rati

In [8]:
# ## 8. 결과 저장 (JSON 직렬화) 테스트

logger.info("결과 저장 (JSON 직렬화) 테스트 시작")

if result_data is not None:
    try:
        # 결과 저장 디렉토리 확인 및 생성
        ensure_dir(results_dir)
        filepath = os.path.join(results_dir, f"{result_filename_base}.json")

        logger.debug(f"결과를 JSON 파일로 저장 시도: {filepath}")
        with open(filepath, 'w', encoding='utf-8') as f: # 인코딩 명시
            # indent=4로 가독성 높게 저장, ensure_ascii=False로 한글 등 유지
            json.dump(result_data, f, indent=4, ensure_ascii=False)

        logger.info(f"결과가 성공적으로 '{filepath}'에 저장되었습니다.")
        print(f"\n결과 파일이 '{filepath}'에 저장되었습니다.")

        # 저장된 파일 다시 읽어서 확인 (옵션)
        logger.debug("저장된 JSON 파일 유효성 검사...")
        with open(filepath, 'r', encoding='utf-8') as f:
            loaded_data = json.load(f)
        logger.info("저장된 JSON 파일을 성공적으로 다시 로드했습니다.")
        # print("\n저장된 파일 내용 확인 (Metrics):")
        # print(json.dumps(loaded_data.get('metrics', {}), indent=2, ensure_ascii=False))


    except TypeError as e:
        # 이 오류는 보통 result_data 내부에 JSON으로 변환할 수 없는 타입이 남아있을 때 발생
        logger.exception(f"JSON 직렬화 중 TypeError 발생: {e}")
        print(f"\nJSON 저장 실패: 데이터 타입 오류 가능성.")
        # 오류 발생 시 result_data의 각 항목 타입 확인
        print("Result Data Types:")
        for k, v in result_data.items():
             if k == 'metrics' and isinstance(v, dict):
                 print("  Metrics Types:")
                 for mk, mv in v.items():
                     print(f"    {mk}: {type(mv)}")
             elif k == 'parameters' and isinstance(v, dict):
                  print("  Parameters Types:")
                  for pk, pv in v.items():
                      print(f"    {pk}: {type(pv)}")
             else:
                 print(f"  {k}: {type(v)}")

    except Exception as e:
        logger.exception(f"결과 저장 중 예상치 못한 오류 발생: {e}")

else:
    logger.error("결과 데이터(result_data)가 생성되지 않아 저장을 건너<0xEB><0x9B><0x84>니다.")

print("\n--- 디버깅 완료 ---")

2025-04-17 14:15:22,338 - INFO - 결과 저장 (JSON 직렬화) 테스트 시작
2025-04-17 14:15:22,339 - INFO - 결과가 성공적으로 '/Users/hjkim/Dev/Hjkim/Trading/mcp/results_debug/debug_BTC-USD_2023-07-01_2023-12-31_1h_atr_sl_multiplier1.5_atr_tp_multiplier2.0_pullback_threshold0.05_rsi_period10_sma_long_period20_sma_short_period7.json'에 저장되었습니다.
2025-04-17 14:15:22,340 - INFO - 저장된 JSON 파일을 성공적으로 다시 로드했습니다.



결과 파일이 '/Users/hjkim/Dev/Hjkim/Trading/mcp/results_debug/debug_BTC-USD_2023-07-01_2023-12-31_1h_atr_sl_multiplier1.5_atr_tp_multiplier2.0_pullback_threshold0.05_rsi_period10_sma_long_period20_sma_short_period7.json'에 저장되었습니다.

--- 디버깅 완료 ---


In [10]:
# ## 9. 백테스트 결과 시각화 (재수정)

if portfolio is not None:
    logger.info("백테스트 결과 시각화 시도...")
    try:
        # --- 거래 기록 컬럼 확인 ---
        if not portfolio.trades.records.empty:
            print("\n--- Trade Records Columns ---")
            print(portfolio.trades.records.columns.tolist())
            # --> 이전 실행 결과에서 'entry_price' 확인됨
        else:
            print("\n--- No trades executed, skipping trade records info. ---")

        # --- 기본 플롯 생성 (잘못된 파라미터 제거) ---
        logger.info("Generating basic portfolio plot...")
        # plot_drawdowns=True 인자 제거
        fig = portfolio.plot(
             # plot_orders=True, # 필요시 주석 해제
             # plot_trade_map=False # 하단 거래 맵 제외 (옵션)
        )
        fig.show()
        logger.info("백테스트 기본 플롯 생성 완료.")

        # --- (옵션) SL/TP 시각화 부분 ---
        # logger.warning("SL/TP level visualization is currently disabled.")
        # ... (이전 코드 참고, 'entry_price' 컬럼 사용) ...

    except Exception as e:
        logger.exception(f"백테스트 결과 시각화 중 오류 발생: {e}")
else:
    logger.warning("포트폴리오 객체가 없어 시각화를 건너뜁니다.")


2025-04-17 14:15:39,204 - INFO - 백테스트 결과 시각화 시도...
2025-04-17 14:15:39,206 - INFO - Generating basic portfolio plot...



--- Trade Records Columns ---
['id', 'col', 'size', 'entry_idx', 'entry_price', 'entry_fees', 'exit_idx', 'exit_price', 'exit_fees', 'pnl', 'return', 'direction', 'status', 'parent_id']


2025-04-17 14:15:39,545 - INFO - 백테스트 기본 플롯 생성 완료.
